Data Cleaning

In [152]:
#Environment Setup

import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore')

In [153]:
credit_df = pd.read_csv('Data\Loan Data\loan_data_2007_2014.csv')
credit_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
# Handling Null Values

credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           466285 non-null  int64  
 1   member_id                    466285 non-null  int64  
 2   loan_amnt                    466285 non-null  int64  
 3   funded_amnt                  466285 non-null  int64  
 4   funded_amnt_inv              466285 non-null  float64
 5   term                         466285 non-null  object 
 6   int_rate                     466285 non-null  float64
 7   installment                  466285 non-null  float64
 8   grade                        466285 non-null  object 
 9   sub_grade                    466285 non-null  object 
 10  emp_title                    438697 non-null  object 
 11  emp_length                   445277 non-null  object 
 12  home_ownership               466285 non-null  object 
 13 

In [155]:
null_vals = credit_df.isnull().mean()
null_vals[null_vals >0.5]

# Dropping only columns with no values at all eventhough some columns have more than 70% null values 
# This is because not all features are applicable to all customers.

desc                           0.729815
mths_since_last_delinq         0.536906
mths_since_last_record         0.865666
mths_since_last_major_derog    0.787739
annual_inc_joint               1.000000
dti_joint                      1.000000
verification_status_joint      1.000000
open_acc_6m                    1.000000
open_il_6m                     1.000000
open_il_12m                    1.000000
open_il_24m                    1.000000
mths_since_rcnt_il             1.000000
total_bal_il                   1.000000
il_util                        1.000000
open_rv_12m                    1.000000
open_rv_24m                    1.000000
max_bal_bc                     1.000000
all_util                       1.000000
inq_fi                         1.000000
total_cu_tl                    1.000000
inq_last_12m                   1.000000
dtype: float64

In [156]:
# Deleting columns with no values

null_cols = null_vals[null_vals ==1.0]
credit_df.drop(columns = null_cols.index, inplace = True)

In [157]:
# Deleting columns not required for the model

credit_df.drop(columns = ['id', 'member_id', 'sub_grade', 'emp_title', 'url', 'desc', 'earliest_cr_line', 'last_credit_pull_d',
                          'purpose', 'title', 'zip_code', 'addr_state','collection_recovery_fee', 'policy_code', 'application_type'], inplace = True)

In [158]:
# Extracting numeric information from variables by stripping off the strings

credit_df['term'] = pd.to_numeric(credit_df['term'].str.replace('months', ''))

credit_df['emp_length'] = credit_df['emp_length'].str.replace('< 1 year', '1')
replacements = '|'.join(['\+ years', ' years', ' year'])
credit_df['emp_length'] = credit_df['emp_length'].str.replace(replacements, '')

credit_df['emp_length'] = pd.to_numeric(credit_df['emp_length'])

In [159]:
# Imputation

credit_df['emp_length'].fillna(value=0, inplace=True)
credit_df['emp_length'].isna().sum()

0

In [160]:
credit_df['last_pymnt_d'].loc[:].isnull().sum()

376

In [161]:
# While the columns being dropped hold information, they are being dropped because either they have high null values or imputation would make it applicable to all customers which is not possible

credit_df.drop(columns=['mths_since_last_delinq', 'mths_since_last_record', 'revol_util', 'next_pymnt_d', 'mths_since_last_major_derog', 'tot_cur_bal' , 'total_rev_hi_lim'], inplace=True)

'''
For example: for no record of 'mths_since_last_delinq' (months since last delinquency), imputation with Value = 0 would mean the customers' latest delinqunecy happened in the current month. 
Imputation with a central tendency would be incompatible with data of customers whose loans are already paid or charged of
'''

credit_df.dropna(subset=['last_pymnt_d'], inplace=True)

# Null values in 'last_pymnt_d' (Last payment date) is low and therefore we drop the rows with no last payment dat and retain the valuable information

In [162]:
# Imputation of columns that are logically applicable to all customers

credit_df.fillna({'delinq_2yrs':0, 'inq_last_6mths':0, 'open_acc':0, 'pub_rec':0, 'tot_coll_amt':0, 'acc_now_delinq':0, 'collections_12_mths_ex_med':0}, inplace=True)
credit_df['total_acc'].fillna(value=round(credit_df['total_acc'].mean(),0), inplace=True)
credit_df['annual_inc'].fillna(value=round(credit_df['annual_inc'].mean(),2), inplace=True)

In [163]:
# Handling categorical data 

columns_list = ['grade', 'home_ownership', 'verification_status', 'pymnt_plan', 'initial_list_status']
df_dummies = []

for col in columns_list:
    df_dummies.append(pd.get_dummies(credit_df[col], prefix = col, prefix_sep = ': '))

df_dummies = pd.concat(df_dummies, axis = 1)
credit_df = pd.concat([credit_df, df_dummies], axis = 1)


In [164]:
credit_df = credit_df.drop(columns=columns_list)

In [165]:
credit_df['last_pymnt_d']

0         Jan-15
1         Apr-13
2         Jun-14
3         Jan-15
4         Jan-16
           ...  
466280    Jan-16
466281    Dec-14
466282    Jan-16
466283    Dec-14
466284    Jan-16
Name: last_pymnt_d, Length: 465909, dtype: object

In [166]:
# Handling date columns 

date_columns = ['issue_d', 'last_pymnt_d']

for i in date_columns:
    credit_df[i] = pd.to_datetime(credit_df[i], format='%b-%y')


In [167]:
# Calculating the duration of customer bank credit relationship 

credit_df['punctuality'] = round(pd.to_numeric((credit_df['last_pymnt_d'] - credit_df['issue_d']) / np.timedelta64(1, 'M')))

In [168]:
credit_df = credit_df.drop(columns=date_columns)

In [169]:
# Preparing target Variable

credit_df['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [170]:
credit_df['Non Performing'] = np.where(credit_df.loc[:, 'loan_status'].isin(['Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)', 'In Grace Period',
                                                               'Does not meet the credit policy. Status:Charged Off']), 1, 0)


In [171]:
credit_df = credit_df.drop(columns=['loan_status'])

In [172]:
credit_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,...,home_ownership: RENT,verification_status: Not Verified,verification_status: Source Verified,verification_status: Verified,pymnt_plan: n,pymnt_plan: y,initial_list_status: f,initial_list_status: w,punctuality,Non Performing
0,5000,5000,4975.0,36,10.65,162.87,10.0,24000.0,27.65,0.0,...,1,0,0,1,1,0,1,0,37.0,0
1,2500,2500,2500.0,60,15.27,59.83,1.0,30000.0,1.00,0.0,...,1,0,1,0,1,0,1,0,16.0,1
2,2400,2400,2400.0,36,15.96,84.33,10.0,12252.0,8.72,0.0,...,1,1,0,0,1,0,1,0,30.0,0
3,10000,10000,10000.0,36,13.49,339.31,10.0,49200.0,20.00,0.0,...,1,0,1,0,1,0,1,0,37.0,0
4,3000,3000,3000.0,60,12.69,67.79,1.0,80000.0,17.94,0.0,...,1,0,1,0,1,0,1,0,49.0,0


In [173]:
credit_df.isnull().sum()

loan_amnt                               0
funded_amnt                             0
funded_amnt_inv                         0
term                                    0
int_rate                                0
installment                             0
emp_length                              0
annual_inc                              0
dti                                     0
delinq_2yrs                             0
inq_last_6mths                          0
open_acc                                0
pub_rec                                 0
revol_bal                               0
total_acc                               0
out_prncp                               0
out_prncp_inv                           0
total_pymnt                             0
total_pymnt_inv                         0
total_rec_prncp                         0
total_rec_int                           0
total_rec_late_fee                      0
recoveries                              0
last_pymnt_amnt                   

In [174]:
credit_df.to_csv('Data/Loan Data/Cleaned_Data.csv')

With no null values and target variable treated, the cleaned dataset is ready for model implementation